In [1]:
library(tidyverse)
library(survey)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: grid

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: survival


Attaching package: ‘survey’


The following object is masked from ‘package:graphics’:

    dotchart




# Data ingestion

In [2]:
data_raw <- read.csv('adult23csv/adult23.csv', stringsAsFactors=FALSE)
head(data_raw)

,URBRRL,RATCAT_A,INCTCFLG_A,IMPINCFLG_A,LANGSPECR_A,LANGSOC_A,LANGDOC_A,LANGMED_A,LANGHM_A,PPSU,⋯,PROXYREL_A,PROXY_A,AVAIL_A,HHSTAT_A,INTV_MON,RECTYPE,IMPNUM_A,WTFA_A,HHX,POVRATTC_A
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<chr>,<dbl>
1,3,4,0,0,NA,NA,NA,NA,NA,2,⋯,NA,NA,1,1,1,10,1,7371.139,H029691,1.01
2,4,8,0,0,NA,NA,NA,NA,NA,2,⋯,NA,NA,1,1,1,10,1,3146.794,H028812,2.49
3,4,14,0,0,NA,NA,NA,NA,NA,2,⋯,NA,NA,1,1,1,10,1,10807.558,H045277,6.73
4,4,10,0,0,NA,NA,NA,NA,NA,2,⋯,NA,NA,1,1,1,10,1,4661.643,H021192,3.43
5,4,5,0,0,NA,NA,NA,NA,NA,2,⋯,NA,NA,1,1,1,10,1,10929.554,H025576,1.27
6,4,14,0,0,NA,NA,NA,NA,NA,2,⋯,NA,NA,1,1,1,10,1,3613.145,H010713,9.18


In [3]:
data = data_raw %>%
    select(
        AGEP_A, SEX_A, HISPALLP_A, REGION, SMKEV_A, RATCAT_A, EMPLASTWK_A, REPSTRAIN_A,
        WTFA_A, PSTRAT, PPSU
    ) %>%
    rename(
        age = AGEP_A,
        sex = SEX_A,
        race_eth = HISPALLP_A,
        region = REGION,
        ever_smoker = SMKEV_A,
        poverty = RATCAT_A,
        employed = EMPLASTWK_A,
        repstrain = REPSTRAIN_A,

        .weight = WTFA_A,
        .pseudo_stratum = PSTRAT,
        .pseudo_psu = PPSU,
    ) %>%
    mutate(
        age = relevel(factor(case_when(
            age < 18 ~ "<18",
            (age >= 18 & age <= 24) ~ "18-24",
            (age >= 25 & age <= 44) ~ "25-44",
            (age >= 45 & age <= 64) ~ "45-64",
            age >= 65 ~ "65+",
            TRUE ~ NA
        )), ref = "25-44"),
        sex = factor(case_when(
            sex == 1 ~ "Male",
            sex == 2 ~ "Female",
            TRUE ~ NA
        )),
        race_eth = relevel(factor(case_when(
            race_eth == 1 ~ "Hispanic",
            race_eth == 2 ~ "Non-Hispanic White",
            race_eth == 3 ~ "Non-Hispanic Black",
            race_eth == 4 ~ "Non-Hispanic Asian",
            race_eth %in% c(5, 6, 7) ~ "Non-Hispanic Other",
            TRUE ~ NA
        )), ref = "Non-Hispanic White"),
        region = relevel(factor(case_when(
            region == 1 ~ "Northeast",
            region == 2 ~ "Midwest",
            region == 3 ~ "South",
            region == 4 ~ "West",
            TRUE ~ NA
        )), ref = "Northeast"),
        ever_smoker = case_when(
            ever_smoker %in% c(1) ~ TRUE,
            ever_smoker %in% c(2) ~ FALSE,
            TRUE ~ NA
        ),
        poverty = case_when(
            poverty %in% c(1, 2, 3) ~ TRUE,
            poverty %in% c(4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14) ~ FALSE,
            TRUE ~ NA
        ),
        employed = case_when(
            employed == 1 ~ TRUE,
            employed == 2 ~ FALSE,
            TRUE ~ NA
        ),
        repstrain = case_when(
            repstrain == 1 ~ TRUE,
            repstrain == 2 ~ FALSE,
            TRUE ~ NA
        )
    )
    
data %>% head()

,age,sex,race_eth,region,ever_smoker,poverty,employed,repstrain,.weight,.pseudo_stratum,.pseudo_psu
,<fct>,<fct>,<fct>,<fct>,<lgl>,<lgl>,<lgl>,<lgl>,<dbl>,<int>,<int>
1,65+,Male,Non-Hispanic Black,South,FALSE,FALSE,FALSE,FALSE,7371.139,103,2
2,65+,Male,Non-Hispanic White,South,TRUE,FALSE,FALSE,FALSE,3146.794,122,2
3,45-64,Male,Non-Hispanic Black,South,FALSE,FALSE,TRUE,FALSE,10807.558,122,2
4,25-44,Female,Non-Hispanic White,South,TRUE,FALSE,TRUE,FALSE,4661.643,122,2
5,45-64,Female,Non-Hispanic White,South,FALSE,FALSE,TRUE,FALSE,10929.554,122,2
6,45-64,Female,Non-Hispanic White,South,FALSE,FALSE,FALSE,FALSE,3613.145,122,2


In [4]:
design = svydesign(
    id = ~.pseudo_psu,
    strata = ~.pseudo_stratum,
    weights = ~.weight,
    data = data,
    nest = TRUE
)

summary(design)

Stratified 1 - level Cluster Sampling design (with replacement)
With (662) clusters.
svydesign(id = ~.pseudo_psu, strata = ~.pseudo_stratum, weights = ~.weight, 
    data = data, nest = TRUE)
Probabilities:
     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
2.505e-05 9.095e-05 1.356e-04 1.632e-04 2.154e-04 5.579e-04 
Stratum Sizes: 
           100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
obs        853 696 579 437 546 551 634 790 844 691 549 510 129 843 387 540 190
design.PSU  17  19  16  12  11  12  14  19  17  16  15   9   3  16  10  14   7
actual.PSU  17  19  16  12  11  12  14  19  17  16  15   9   3  16  10  14   7
           117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
obs        635 591 797 494 627 557 276 535 358 601 466 303 782 526 583 538 589
design.PSU  15  14  14  12  11  16   7  14  10  15  10   6  13  12  14  12  14
actual.PSU  15  14  14  12  11  16   7  14  10  15  10   6  13  12  14  12  14
           134 135 136 

In [5]:
write.csv(data, "out/data.csv", row.names = FALSE)

# Data subsetting

In [6]:
design_employed = subset(
    design,
    employed == TRUE,
    !is.na(repstrain),
    !is.na(sex),
    !is.na(ever_smoker)
)
design_employed$variables$employed = NULL

design_employed

Stratified 1 - level Cluster Sampling design (with replacement)
With (660) clusters.
subset(design, employed == TRUE, !is.na(repstrain), !is.na(sex), 
    !is.na(ever_smoker))

In [7]:
nrow(data)
nrow(data %>% filter(!is.na(repstrain)))
nrow(data %>% filter(employed == TRUE, !is.na(repstrain)))
nrow(data %>% filter(
    employed == TRUE,
    !is.na(repstrain),
    !is.na(sex),
    !is.na(ever_smoker)
))

modeling_data = data %>% filter(
    employed == TRUE,
    !is.na(repstrain),
    !is.na(sex),
    !is.na(ever_smoker)
)

[1] 29522

[1] 28550

[1] 15621

[1] 15596

# Table 1

In [8]:
outcome_var = "repstrain"

comp_cols = colnames(design_employed)[
    !startsWith(colnames(design_employed), ".") &
    !(colnames(design_employed) == outcome_var)
]

comp_cols

[1] "age"         "sex"         "race_eth"    "region"      "ever_smoker"
[6] "poverty"

In [9]:
# generate a table 1 for a single variable

t1_var = function(vname) {
    weighted = data.frame(
        svytable(
            as.formula(paste0("~", vname, " + ", outcome_var)),
            design_employed,
            na.rm = FALSE,
            round = TRUE
        )) %>%
        mutate(weighted = "weighted")

    unweighted = modeling_data %>%
        group_by(!!sym(outcome_var), !!sym(vname)) %>%
        summarise(Freq = n(), .groups = "drop") %>%
        mutate(
            weighted = "unweighted",
            !!sym(outcome_var) := factor(!!sym(outcome_var)),
            !!sym(vname) := factor(.data[[vname]])
        )
    

    bind_rows(weighted, unweighted) %>%
        mutate(
            variable = vname,
            outcome_var := factor(case_when(
                !!sym(outcome_var) == TRUE ~ "outcome",
                !!sym(outcome_var) == FALSE ~ "no_outcome",
                TRUE ~ NA
            ))
        ) %>%
        rename(value = !!sym(vname), n = Freq) %>%
        select(variable, value, everything(), -!!sym(outcome_var)) %>%
        pivot_wider(names_from = outcome_var, values_from = n) %>%
        mutate(
            Overall = outcome + no_outcome,
        ) %>%
        pivot_longer(
            cols = c(outcome, no_outcome, Overall),
            names_to = outcome_var,
            values_to = "n"
        ) %>%
        pivot_wider(names_from = c(weighted, !!sym(outcome_var)), values_from = n) %>%
        select(variable, value, everything())
}

# demo the function
t1_var("age")

variable,value,weighted_outcome,weighted_no_outcome,weighted_Overall,unweighted_outcome,unweighted_no_outcome,unweighted_Overall
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
age,25-44,6449067,59668991,66118058,686,6026,6712
age,18-24,1204475,17957079,19161554,79,1139,1218
age,45-64,6122200,48320319,54442519,711,5330,6041
age,65+,802832,9394392,10197224,135,1490,1625


In [10]:
# generate table 1 for all variables
t1_res = list()
for (vname in comp_cols) {
    t1_res[[vname]] = t1_var(vname)
}

t1_res = bind_rows(t1_res) %>%
    mutate(
        variable = ifelse(!is.na(lag(variable)) & (variable == lag(variable)), "", variable),
        w_pct_outcome = paste0(round(weighted_outcome / weighted_Overall, 4) * 100, "%"),
        w_pct_no_outcome = paste0(round(weighted_no_outcome / weighted_Overall, 4) * 100, "%")
    ) %>%
    select(
        variable, value,
        unweighted_Overall, weighted_Overall,
        unweighted_outcome, weighted_outcome, w_pct_outcome,
        unweighted_no_outcome, weighted_no_outcome, w_pct_no_outcome
        )
t1_res

variable,value,unweighted_Overall,weighted_Overall,unweighted_outcome,weighted_outcome,w_pct_outcome,unweighted_no_outcome,weighted_no_outcome,w_pct_no_outcome
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
age,25-44,6712,66118058,686,6449067,9.75%,6026,59668991,90.25%
,18-24,1218,19161554,79,1204475,6.29%,1139,17957079,93.71%
,45-64,6041,54442519,711,6122200,11.25%,5330,48320319,88.75%
,65+,1625,10197224,135,802832,7.87%,1490,9394392,92.13%
sex,Female,7745,69460559,823,7062455,10.17%,6922,62398104,89.83%
,Male,7851,80408263,788,7516120,9.35%,7063,72892143,90.65%
race_eth,Non-Hispanic White,9850,90568054,1088,9635016,10.64%,8762,80933038,89.36%
,Hispanic,2665,27957539,227,2162114,7.73%,2438,25795425,92.27%
,Non-Hispanic Asian,1039,10339886,101,951249,9.2%,938,9388637,90.8%


Do $\chi^2$ tests for all variables and put the p-values in a table.

In [11]:
p_res = list()
for (vname in comp_cols) {    
    test = svychisq(
        as.formula(paste0("~ ", outcome_var, " + ", vname)),
        design = design_employed,
        statistic = "Chisq",
        na.rm = TRUE
    )

    p_res[[vname]] = data.frame(
        variable = vname,
        p_value = unname(test$p.value),
        test_name = test$method
    )
}

p_res = bind_rows(p_res) %>%
    mutate(
        p_value = case_when(
            p_value < 0.001 ~ "<0.001*",
            p_value <= 0.05 & p_value >= 0.001 ~ paste0(round(p_value, 3), "*"),
            p_value >= 0.001 ~ as.character(round(p_value, 3))
        )
    )
p_res

variable,p_value,test_name
<chr>,<chr>,<chr>
age,<0.001*,Pearson's X^2: Rao & Scott adjustment
sex,0.121,Pearson's X^2: Rao & Scott adjustment
race_eth,0.002*,Pearson's X^2: Rao & Scott adjustment
region,0.514,Pearson's X^2: Rao & Scott adjustment
ever_smoker,<0.001*,Pearson's X^2: Rao & Scott adjustment
poverty,0.228,Pearson's X^2: Rao & Scott adjustment


In [12]:
# combine p-values with descriptive statistics to create final table 1
t1_final = t1_res %>%
    left_join(p_res, by = "variable", ) %>%
    replace_na(list(p_value = "", test_name = ""))

t1_final

variable,value,unweighted_Overall,weighted_Overall,unweighted_outcome,weighted_outcome,w_pct_outcome,unweighted_no_outcome,weighted_no_outcome,w_pct_no_outcome,p_value,test_name
<chr>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
age,25-44,6712,66118058,686,6449067,9.75%,6026,59668991,90.25%,<0.001*,Pearson's X^2: Rao & Scott adjustment
,18-24,1218,19161554,79,1204475,6.29%,1139,17957079,93.71%,,
,45-64,6041,54442519,711,6122200,11.25%,5330,48320319,88.75%,,
,65+,1625,10197224,135,802832,7.87%,1490,9394392,92.13%,,
sex,Female,7745,69460559,823,7062455,10.17%,6922,62398104,89.83%,0.121,Pearson's X^2: Rao & Scott adjustment
,Male,7851,80408263,788,7516120,9.35%,7063,72892143,90.65%,,
race_eth,Non-Hispanic White,9850,90568054,1088,9635016,10.64%,8762,80933038,89.36%,0.002*,Pearson's X^2: Rao & Scott adjustment
,Hispanic,2665,27957539,227,2162114,7.73%,2438,25795425,92.27%,,
,Non-Hispanic Asian,1039,10339886,101,951249,9.2%,938,9388637,90.8%,,


In [13]:
write.csv(t1_final, "out/t1_final.csv", row.names = FALSE)

# Modeling

In [14]:
m1 = svyglm(repstrain ~ age + sex + race_eth + region + poverty + ever_smoker, design = design_employed, family = "quasibinomial")
summary(m1)


Call:
svyglm(formula = repstrain ~ age + sex + race_eth + region + 
    poverty + ever_smoker, design = design_employed, family = "quasibinomial")

Survey design:
subset(design, employed == TRUE, !is.na(repstrain), !is.na(sex), 
    !is.na(ever_smoker))

Coefficients:
                           Estimate Std. Error t value Pr(>|t|)    
(Intercept)                -2.20892    0.09686 -22.806  < 2e-16 ***
age18-24                   -0.40334    0.14863  -2.714 0.006846 ** 
age45-64                    0.12260    0.06395   1.917 0.055700 .  
age65+                     -0.30553    0.11009  -2.775 0.005690 ** 
sexMale                    -0.12601    0.06079  -2.073 0.038606 *  
race_ethHispanic           -0.33464    0.10067  -3.324 0.000941 ***
race_ethNon-Hispanic Asian -0.15651    0.13511  -1.158 0.247187    
race_ethNon-Hispanic Black -0.17695    0.10725  -1.650 0.099480 .  
race_ethNon-Hispanic Other -0.24322    0.23259  -1.046 0.296114    
regionMidwest              -0.03446    0.10793  -0

In [15]:
m1_coef = data.frame(summary(m1)$coefficients) %>%
    mutate(
        `Odds Ratio` = exp(Estimate),
        `p value` = `Pr...t..`,
    ) %>%
    rownames_to_column(var = "Term") %>%
    mutate(
        `Odds Ratio` = round(`Odds Ratio`, 3),
        `p value` = case_when(
            `p value` < 0.001 ~ "<0.001*",
            `p value` <= 0.05 & `p value` >= 0.001 ~ paste0(round(`p value`, 3), "*"),
            `p value` > 0.05 ~ as.character(round(`p value`, 3))
        ),
        ci_low = exp(Estimate - 1.96 * `Std..Error`),
        ci_high = exp(Estimate + 1.96 * `Std..Error`),
        `95% Confidence Interval` = paste0("(", round(ci_low, 3), ", ", round(ci_high, 3), ")")
    ) %>%
    select(Term, `Odds Ratio`, `95% Confidence Interval`, `p value`)

m1_coef

write.csv(m1_coef, "out/m1_coef.csv", row.names = FALSE)

Term,Odds Ratio,95% Confidence Interval,p value
<chr>,<dbl>,<chr>,<chr>
(Intercept),0.110,"(0.091, 0.133)",<0.001*
age18-24,0.668,"(0.499, 0.894)",0.007*
age45-64,1.130,"(0.997, 1.281)",0.056
age65+,0.737,"(0.594, 0.914)",0.006*
sexMale,0.882,"(0.783, 0.993)",0.039*
race_ethHispanic,0.716,"(0.587, 0.872)",<0.001*
race_ethNon-Hispanic Asian,0.855,"(0.656, 1.114)",0.247
race_ethNon-Hispanic Black,0.838,"(0.679, 1.034)",0.099
race_ethNon-Hispanic Other,0.784,"(0.497, 1.237)",0.296
